# Summary
This notebook is used to automatically extract data from ATB yearly workbook file and generate summary accordingly to the specification.

## Getting Ready
- Here, we first import the package needed, and identify the path to files;
    - 2 paths to files are required here:
        1. raw_data_path: the path to the raw data file
        2. ancillary_path: the path to the ancillary file which declares the scope of tech and index of interest.

- Make 2 lists out of the ancillary file

In [105]:
import pandas as pd

In [106]:
raw_data_path = '/Users/zhixuan/PycharmProjects/ATB-Raw-Summarization/data/2022 v2 Annual Technology Baseline Workbook Corrected 7-21-2022.xlsx'

ancillary_path = '/Users/zhixuan/PycharmProjects/ATB-Raw-Summarization/data/ancillary.xlsx'

raw = pd.ExcelFile(raw_data_path)

In [107]:
ancillary = pd.read_excel(ancillary_path)

# make tech list out of ancillary
tech_list = list(ancillary['Tech'].values)

# make index list out of ancillary
index_list = list(ancillary['Index'].dropna().values)

# flush-flush
del ancillary

## Main loop
- Iterate over tech sheets needed

In [108]:
# get all the name of the sheets -> intersect with tech list
sheet_list = list(set(raw.sheet_names) & set(tech_list))
sub_tech_list = list(set(tech_list)-set(sheet_list))

In [112]:
# iterate through all sheets level techs
sheets = {}
for sheet in sheet_list:
    sheet_df = pd.read_excel(raw, sheet_name=str(sheet)).iloc[:, 9:].dropna(how='all')
    index = None
    area_head = None
    # iterate over rows
    for i in range(len(sheet_df)):
        # get the value on the current index column
        working_index = sheet_df.iloc[i, 0]

        # if it has null value
        if pd.isna(working_index):
            if not index is None:
                # assign the current index to the cell
                sheet_df.iloc[i, 0] = str(index)
        else:
            # first valid
            if index is None and (working_index in index_list):
                print(working_index)
                area_head = i

            if area_head is not None:
                index = str(working_index)

    sheet_df = sheet_df.iloc[area_head:, :]
    sheets[str(sheet)] = sheet_df

Overnight Capital Cost ($/kW)
Overnight Capital Cost ($/kW)
Fixed Operation and Maintenance Expenses ($/kW-yr)
Variable Operation and Maintenance Expenses ($/MWh)
Grid Connection Costs (GCC) ($/kW)
Offshore Spur Line Costs ($/kW)
Levelized Cost of Energy ($/MWh)
Tax Credit
Depreciation Factor
Construction Finance Factor
Overnight Capital Cost ($/kW)
Overnight Capital Cost ($/kW)
Fixed Operation and Maintenance Expenses ($/kW-yr)
Variable Operation and Maintenance Expenses ($/MWh)
Grid Connection Costs (GCC) ($/kW)
Levelized Cost of Energy ($/MWh)
Tax Credit
Depreciation Factor
Construction Finance Factor
Overnight Capital Cost ($/kW)
Overnight Capital Cost ($/kW)
Fixed Operation and Maintenance Expenses ($/kW-yr)
Variable Operation and Maintenance Expenses ($/MWh)
Grid Connection Costs (GCC) ($/kW)
Levelized Cost of Energy ($/MWh)
Tax Credit
Depreciation Factor
Construction Finance Factor
Overnight Capital Cost ($/kW)
Overnight Capital Cost ($/kW)
Fixed Operation and Maintenance Expens

In [110]:
sheets[sheet_list[0]]

,Unnamed: 9,Unnamed: 10,Unnamed: 11,https://atb.nrel.gov/electricity/2022/offshore_wind,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,...,Unnamed: 37,Unnamed: 38,Unnamed: 39,Unnamed: 40,Unnamed: 41,Unnamed: 42,Unnamed: 43,Unnamed: 44,Unnamed: 45,Unnamed: 46
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,Base Year:,NaN,NaN,2020,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,"All values are given in 2020 U.S. dollars, usi...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
579,Construction Finance Factor,NaN,NaN,NaN,"Adapted from Valpy et al., 2017","and Hundleby et al., 2017",Beiter et al. 2020,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
580,Construction Finance Factor,NaN,NaN,NaN,"Adapted from Valpy et al., 2017","and Hundleby et al., 2017",Beiter et al. 2020,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
581,Construction Finance Factor,NaN,NaN,NaN,"Adapted from Valpy et al., 2017","and Hundleby et al., 2019",Beiter et al. 2020,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
584,Construction Finance Factor,NaN,NaN,NaN,"Beiter et al, 2019",Beiter et al. 2020,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [111]:
index_list

['Overnight Capital Cost ($/kW)',
 'Fixed Operation and Maintenance Expenses ($/kW-yr)',
 'Variable Operation and Maintenance Expenses ($/MWh)']